# DENOISING IMAGES WITH AUTOENCODERS

In [1]:
import cv2
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import *

DEFINE THE BUILD_AUTOENCODER() FUNCTION, WHICH CREATES THE CORRESPONDING NEURAL ARCHITECTURE.
    NOTICE THAT THIS IS THE SAME ARCHITECTURE WE IMPLEMENTED I THE PREVIOUS RECIPE; THEREFORE, WE WON'T 
GO INTO TOO MUCH DETAIL HERE. FOR AN IN-DEPTH EXPLANATION, PLEASE REFER TO THE "CREATING AUTOENCODER" RECIPE

In [2]:
def build_autoencoder(input_shape = (28, 28, 1),
                     encoding_size = 128,
                     alpha = 0.2):
    inputs = Input(shape = input_shape)
    encoder = Conv2D(filters = 32,
                    kernel_size = (3, 3),
                    strides = 2,
                    padding = "same") (inputs)
    encoder = LeakyReLU(negative_slope = 0.3) (encoder)
    encoder = BatchNormalization() (encoder)
    encoder = Conv2D(filters = 64,
                    kernel_size = (3, 3),
                    strides = 2,
                    padding = "same") (encoder)
    encoder = LeakyReLU(negative_slope = 0.3) (encoder)
    encoder = BatchNormalization() (encoder)

    encoder_output_shape = encoder.shape
    encoder = Flatten() (encoder)
    encoder_output = Dense(units = encoding_size) (encoder)

    encoder_model = Model(inputs, encoder_output)


    # LET'S CREATE THE DECODER
    decoder_input = Input(shape = (encoding_size, ))
    target_shape = tuple(encoder_output_shape[1:])
    decoder = Dense(np.prod(target_shape)) (decoder_input)

    decoder = Reshape(target_shape) (decoder)

    decoder = Conv2DTranspose(filters = 64,
                             kernel_size = (3, 3),
                             strides = 2,
                             padding = "same") (decoder)
    decoder = LeakyReLU(negative_slope = 0.3) (decoder)
    decoder = BatchNormalization() (decoder)
    
    decoder = Conv2DTranspose(filters = 32,
                             kernel_size = (3, 3),
                             strides = 2,
                             padding = "same") (decoder)
    decoder = LeakyReLU(negative_slope = 0.3) (decoder)
    decoder = BatchNormalization() (decoder)
    
    decoder = Conv2DTranspose(filters = 1,
                             kernel_size = (3, 3),
                             padding = "same") (decoder)
    outputs = Activation("sigmoid") (decoder)
    
    decoder_model = Model(decoder_input, outputs)

    # DEFINE THE ENCODER ITSELF
    encoder_model_output = encoder_model(inputs)
    decoder_model_output = decoder_model(encoder_model_output)
    autoencoder_model = Model(inputs,
                             decoder_model_output)

    return encoder_model, decoder_model, autoencoder_model
    

DEFINE THE PLOT_ORIGINAL_VS_GENERATED() FUNCTION, WHICH CREATES A COMPARATIVE MOSAIC OF THE ORIGINAL AND GENERATED
IMAGES.
WE WILL USE THIS FUNCTION LATER TO SHOW THE NOISY IMAGES AND THEIR RESTORED COUNTERPARTS.
SIMILAR TO BUILD_AUTOENCODER(), THIS FUNCTION WORKS IN THE SAME WAY WE DEFINED IT IN THE "CREATING A SIMPLE FULLY CONNECTED AUTOENCODER" 

In [3]:
def plot_original_vs_generated(original, generated):
    num_images = 15
    sample = np.random.randint(0, len(original),
                              num_images)

DEFINE AN INNER HELPER FUNCTION THAT WILL STACK A SAMPLE OF IMAGES IN A 3X5 GRID:


In [4]:
    def stack(data):
        images = data[sample]
        return np.vstack([np.hstack(images[:5]),
                         np.hstack(images[5:10]),
                         np.hstack(images[10:15])])
        

DEFINE A FUNCTION THAT WILL PUT CUSTOM TEXT ON TOP OF AN IMAGE, IN A CERTAIN LOCATION

In [5]:
def add_text(image, text, position):
    pt1 = position
    pt2 = (pt1[0] + 10 + (len(text) * 22),
          pt1[1] - 45)
    cv2.rectangle(image,
                 pt1,
                 pt2,
                 (255, 255, 255),
                 -1)
    cv2.putText(image, text,
               position,
               fontFace = cv2.FONT_HERSHEY_SIMPLEX,
               fontScale = 1.3,
               color = (0, 0, 0),
               thickness = 4)

    #cCREATEE THE  MOSAIC WITH BOTH THE ORIGINAL AND THE GENERATED IMAGES, 
    # LABEL EACH SUB-GRID, AND DISPLAY THE RESULT:

    original = stack(original)
    generated = stack(generated)

    mosaic = np.vstack([original,
                       generated])

    mosaic = cv2.resize(mosaic, (860, 860),
                       interpolation = cv2.INTER_AREA)
    mosaic = cv2.cvtColor(mosaic, cv2.COLOR_GRAY2BGR)

    
    add_text(mosaic, "Original", (50, 100))
    add_text(mosaic, "Generated", (50, 520))

    cv2.imshow("Mosaic", mosaic)
    cv2.waitKey(0)

LOAD FASHION-MNIST USING TENSORFLOW'S HANDY FUNCTION.
WE WILL ONLY KEEP THE IMAGES SINCE THE LABELS ARE UNNECESSARY

In [6]:
(x_train, _), (x_test, _) = fashion_mnist.load_data()

NORMALIZE THE IMAGES AND ADD A SINGLE COLOR CHANNEL TO THE USING NP.EXPAND_DIMS():

In [7]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train = np.expand_dims(x_train, axis = -1)
x_test = np.expand_dims(x_test, axis = -1)

GENERATE TWO TENSORS WITH THE SAME DIMENSIONS AS X_TRAIN AND X_TEST, RESPECTIVELY.
    THESE WILL CORRSPOND TO RANDOM GAUSIAN NOISE THAT HAS A MEAN AND STANDARD DEVIATION EQUAL TO 0.5

In [8]:
train_noise = np.random.normal(loc = 0.5, scale = 0.5,
                              size = x_train.shape)
test_noise = np.random.normal(loc = 0.5, scale = 0.5,
                             size = x_test.shape)

PURPOSELY DAMAGE BOTH X_TRAIN AND X_TEST BY ADDING TRAIN_NOISE AND TEST_NOISE, RESPECTIVELY.
    MAKE SURE THAT THE VALUES REMAIN BETWEEN 0 AND 1USING NP.CLIP():

In [9]:
x_train_noisy = np.clip(x_train + train_noise, 0, 1)
x_test_noisy = np.clip(x_test + test_noise, 0, 1)

CREATE THE AUTENCODER AND COMPILE IT.
WE WILL USE "ADAM" AS OUR OPTIMIZER AND "MSE" AS OUR LOSS FUNCTION, GIVEN THAT WE ARE INTERESTED
IN REDUCING THE ERROR INSTEAD OF IMPROVING ACCURACY

In [10]:
_, _, autoencoder = build_autoencoder(encoding_size = 128)
autoencoder.compile(optimizer = "adam", loss = "mse")

FIT THE MODEL FOR 20 EPOCHS, ON BATCHES OF 1024 NOISY IMAGES AT A TIME.
    NOTICE THAT THE FEATURES ARE THE NOISY IMAGES, WHILE THE LABELS OR TARGETA ARE 
THE ORIGINAL ONES, PROR TO BEING DAMAGED:

In [11]:
EPOCHS = 20
BATCH_SIZE = 1024
autoencoder.fit(x_train_noisy, x_train,
               epochs = EPOCHS,
               batch_size = BATCH_SIZE,
               shuffle = True,
               validation_data = (x_test_noisy, x_test))

Epoch 1/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - loss: 0.0550 - val_loss: 0.1059
Epoch 2/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 99s 2s/step - loss: 0.0247 - val_loss: 0.1132
Epoch 3/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - loss: 0.0203 - val_loss: 0.1156
Epoch 4/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - loss: 0.0184 - val_loss: 0.1135
Epoch 5/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - loss: 0.0172 - val_loss: 0.1053
Epoch 6/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - loss: 0.0165 - val_loss: 0.0927
Epoch 7/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - loss: 0.0159 - val_loss: 0.0743
Epoch 8/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - loss: 0.0156 - val_loss: 0.0600
Epoch 9/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - loss: 0.0151 - val_loss: 0.0464
Epoch 10/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - loss: 0.0149 - val_loss: 0.0355
Epoch 11/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - loss: 0.0146 - val_loss: 0.0256
Epoch 12/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - loss: 0.0144 - val_l

MAKE  PREDICTIONS WITH THE TRAINED MODEL.
    RESHAPE BOTH THE NOISY AND GENERATED IMAGES BACK TO 28X28, AND SCALE THEM UP TO THE [0, 255] RANGE:

In [12]:
predictions = autoencoder.predict(x_test)

original_shape = (x_test_noisy.shape[0], 28, 28)
predictions = predictions.reshape(original_shape)
x_test_noisy = x_test_noisy.reshape(original_shape)

predictions = (predictions * 255.0).astype("uint8")
x_test_noisy = (x_test_noisy * 255.0).astype("uint8")


313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step


In [14]:
plot_original_vs_generated(x_test_noisy, predictions)